In [11]:
import pandas as pd
import numpy as np
import torch 
from torch import nn 
import matplotlib.pyplot as plt 


In [12]:
# cuda
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [18]:
data = pd.read_csv("/home/veinmahzy/Implement Deep Learning Paper/Reccurent Neural Network /example:RNN/IMDB Dataset.csv")
data.sample(10)

,review,sentiment
4756,I love this film. It's one of those I can watc...,positive
40345,If you are a bit masochistic and like to waste...,negative
4402,The Ballad of Django is a meandering mess of a...,negative
28149,Story about a widowed father (Claude Rains) br...,positive
14762,I saw this very emotionally painful portrayal ...,positive
12789,This movie was extremely boring. It should lea...,negative
30486,The Matador is better upon reflection because ...,positive
18739,"Oddly, I have very little to say about ""The Bo...",negative
20832,"Lillian Hellman, one of America's most famous ...",positive
21729,"This movie was poorly written, poorly acted an...",negative


In [28]:
X,y = data['review'].values,data['sentiment'].values
print(X)
print(y)

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [29]:
train_split = int (0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:],y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40000, 40000, 10000, 10000)

In [43]:
import re
import numpy as np
from collections import Counter
from nltk.corpus import stopwords

def preprocess_string(s):
    s = re.sub(r"[^\w\s]", '', s)  # Hapus karakter non-kata kecuali spasi
    s = re.sub(r"\s+", '', s)  # Hapus spasi berlebih
    s = re.sub(r"\d", '', s)  # Hapus angka
    return s

def build_onehot_dict(x_train, vocab_size=1000):
    word_list = []
    stop_words = set(stopwords.words('english')) 
    
    for sent in x_train:
        for word in sent.lower().split():
            word = preprocess_string(word)
            if word not in stop_words and word:
                word_list.append(word)
    
    corpus = Counter(word_list)
    common_words = sorted(corpus, key=corpus.get, reverse=True)[:vocab_size]
    onehot_dict = {w: i + 1 for i, w in enumerate(common_words)}
    
    return onehot_dict

def tokenize_sentences(sentences, onehot_dict):

    tokenized_sentences = []
    for sent in sentences:
        tokenized_sentences.append([
            onehot_dict[preprocess_string(word)] 
            for word in sent.lower().split() 
            if preprocess_string(word) in onehot_dict
        ])
    return np.array(tokenized_sentences, dtype=object)

def encode_labels(labels):
    return np.array([1 if label == 'positive' else 0 for label in labels])

def tokenize(x_train, y_train, x_val, y_val, vocab_size=1000):
   
    onehot_dict = build_onehot_dict(x_train, vocab_size)
    x_train_tokens = tokenize_sentences(x_train, onehot_dict)
    x_val_tokens = tokenize_sentences(x_val, onehot_dict)
    y_train_encoded = encode_labels(y_train)
    y_val_encoded = encode_labels(y_val)
    
    return x_train_tokens, y_train_encoded, x_val_tokens, y_val_encoded, onehot_dict


In [36]:
X_train,y_train,X_test,y_test,vocab = tokenize( X_train,y_train,X_test,y_test)

In [40]:
print(X_train)
X_train.shape

[list([4, 941, 57, 299, 335, 105, 472, 479, 1, 20, 58, 51, 462, 180, 105, 559, 53, 42, 42, 276, 462, 248, 234, 1, 361, 235, 958, 413, 295, 213, 413, 237, 220, 34, 131, 1, 8, 47, 170, 42, 546, 93, 163, 157, 442, 698, 85, 956, 698, 698, 60, 869, 89, 20, 299, 43, 107, 292, 47, 177, 91, 213, 462, 462, 383, 503, 15, 141, 14, 646, 686, 541, 546, 433, 790, 440, 57, 101, 309, 15, 390])
 list([274, 39, 250, 1, 1, 288, 401, 178, 316, 296, 1, 1, 59, 430, 14, 386, 91, 244, 11, 678, 14, 165, 57, 322, 296, 250, 4, 16, 113, 44, 1, 1, 10, 160, 237, 39, 84, 961, 142, 234, 893, 188, 446, 51, 593, 446, 964, 73, 14, 126])
 list([100, 274, 27, 959, 9, 774, 822, 631, 57, 113, 38, 298, 28, 7, 14, 382, 101, 570, 812, 919, 124, 100, 46, 323, 31, 77, 1, 340, 4, 62, 47, 96, 34, 105, 719, 90, 1, 101, 539, 115, 16, 113, 53, 11, 225])
 ...
 list([42, 36, 59, 16, 12, 300, 28, 285, 364, 471, 16, 298, 10, 577, 709, 334, 16, 471, 16, 483, 188, 14, 373, 268, 205, 892, 42, 10, 431, 4, 385, 672, 42, 434, 734, 72, 765, 403

(40000,)

In [45]:
from torch.utils.data import TensorDataset, DataLoader
def padding_(sentences, seq_len):
    
    features = np.zeros((len(sentences), seq_len), dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

batch_size = 50

def create_dataloaders(x_train, y_train, x_val, y_val, seq_len=500, batch_size=50):
    """
    Make data loader for validation and train
    """
    x_train_pad = padding_(x_train, seq_len)
    x_val_pad = padding_(x_val, seq_len)
    
    train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
    valid_data = TensorDataset(torch.from_numpy(x_val_pad), torch.from_numpy(y_val))
    
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
    
    return train_loader, valid_loader
